In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core import Settings
from IPython.display import Markdown, display

documents = SimpleDirectoryReader(
    "./input"
).load_data()
print(f"Loaded {len(documents)} documents")

Loaded 12 documents


In [24]:
from llama_index.llms.groq import Groq
from llama_index.llms.ollama import Ollama
from langchain_community.embeddings.ollama import OllamaEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding

ollama_embeddings = OllamaEmbeddings()
embeddings = LangchainEmbedding(ollama_embeddings)
# llm = Ollama(model="llama2", request_timeout=120.0)
llm = Groq(model="llama2-70b-4096")
Settings.llm = llm
Settings.embed_model = embeddings

In [25]:
import os
from llama_index.core import StorageContext, load_index_from_storage

def get_kg_index():
    if os.path.exists("./indices/kg_index"):
        graph_store = SimpleGraphStore()
        kg_storage_ctx = StorageContext.from_defaults(
            persist_dir="./indices/kg_index", graph_store=graph_store)
        kg_index = load_index_from_storage(kg_storage_ctx)
        return kg_index
    else:
        graph_store = SimpleGraphStore()
        storage_context = StorageContext.from_defaults(graph_store=graph_store)
        kg_index = KnowledgeGraphIndex.from_documents(
            documents,
            max_triplets_per_chunk=2,
            storage_context=storage_context,
            include_embeddings=True,
            show_progress=True,
        )
        kg_index.storage_context.persist(persist_dir="./indices/kg_index")
        return kg_index

In [26]:
kg_idx = get_kg_index()

In [27]:
from pyvis.network import Network

g = kg_idx.get_networkx_graph()
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)
net.show("example.html")

example.html


In [15]:
query_engine = kg_idx.as_query_engine(
    embedding_mode="hybrid",
    similarity_top_k=5,
    include_text=True,
    retriever_mode="keyword",
    response_mode="tree_summarize",
)

In [16]:
response = query_engine.query(
    "Explain the ring star problem?",
)

In [17]:
display(Markdown(f"<b>{response}</b>"))

<b>The ring star problem is a classic problem in the field of computer science and algorithms. It is a problem of finding the maximum number of times a subarray of a given array can be repeated in a cyclic manner. In other words, given an array of integers, the problem asks for the maximum number of times a subarray of the given array can be repeated in a cyclic manner, such that the first element of the subarray is equal to the last element of the subarray.

For example, consider the array [1, 2, 3, 4, 5, 6, 7]. A subarray of this array is [2, 3, 4]. This subarray can be repeated in a cyclic manner three times: [2, 3, 4], [4, 5, 6], and [6, 7, 1]. Therefore, the ring star problem for this array is 3.

The problem is known to be NP-hard, which means that there is no known polynomial-time algorithm for solving it exactly for all instances. However, there are approximate algorithms that can find near-optimal solutions in reasonable time. The ring star problem has applications in various fields, such as data compression, pattern recognition, and cryptography.</b>

In [18]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="kg_idx_qe",
            description="Provides information about the Ring Star problem and its evolutionary algorithm solution",
        ),
    )
]

In [19]:
from llama_index.core.agent import ReActAgent
# context = """\
# You are a stock market sorcerer who is an expert on the companies Lyft and Uber.\
#     You will answer questions about Uber and Lyft as in the persona of a sorcerer \
#     and veteran stock market investor.
# """
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    # context=context
)

In [20]:
resp = agent.chat(
    "Explain the ring star problem"
)

Thought: I need to use a tool to help me answer the question.
Action: kg_idx_qe
Action Input: {'input': 'ring star problem', 'num_beams': 5}
Observation: The ring star problem refers to a difficulty in astronomy and astrophysics known as the "ring star problem." It is a long-standing issue that has stumped scientists for years.

The problem is as follows: when a star passes close to a black hole, its light is bent by the strong gravitational field of the black hole, creating a bright ring of light around the black hole known as a "Einstein ring." However, the physics of how this occurs is not well understood.

Scientists have tried to simulate this phenomenon using computer models, but the results have been inconsistent and often inaccurate. The ring star problem has important implications for our understanding of black holes and their behavior, so scientists are eager to find a solution.

One possible solution proposed by some scientists is that the ring star effect could be caused by

In [21]:
print(str(resp))

The ring star problem is a difficulty in astronomy and astrophysics that has stumped scientists for years. It refers to the bending of light by a black hole's strong gravitational field, creating a bright ring of light around the black hole known as an Einstein ring. Despite efforts to simulate this phenomenon using computer models, the results have been inconsistent and often inaccurate. Some scientists propose that the ring star effect could be caused by gravitational lensing, but much more research is needed to fully understand the problem.
